# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [6]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [7]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs

In [8]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

66.70140412386954 -149.26578829674344
85.24734295399037 -18.04794525827313
-60.68480245300977 50.96623692537017
1.316175848137334 -57.54500983769343
-83.21287568806983 -78.22746942181962
-67.47703221624157 142.25528918073354
-35.82161783544666 128.0560606268969
-17.559642682756802 -177.07755772490194
-65.79561720460639 -74.50557519267214
-75.61139476173781 151.3602008944486
-57.38281859396965 -167.68884278383223
-66.11564444528545 90.41805301107615
-89.56589479706331 -87.58526418748929
79.48167296509152 90.92389406077137
-75.92108187855808 -107.67472453286902
88.65191396184994 104.83998655742397
10.096503419491711 72.14084112780276
-43.101751632228805 -105.14101593999655
11.226460914699743 -145.0730918039635
-87.44723994226413 -156.60303188194678
8.670550794316398 -128.0311445392236
-17.996307205675734 60.002030163544475
6.137587661904789 -68.32677592508752
-7.645039097536895 -119.44545582986842
88.97378959474946 83.50099431402708
-73.45077888011834 25.612833627767543
-76.4221967576320

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [9]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

760

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [10]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [11]:
# Assemble the OpenWeatherMap's API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=d6b2e3a4db3dd441dff35baef77ba050


## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [12]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]       
        city_weather_dc = city_weather["weather"][0]["description"]
        # Append the city information into the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_dc})
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | college
Processing Record 2 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 3 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 4 of Set 1 | bonfim
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | hobart
Processing Record 7 of Set 1 | esperance
Processing Record 8 of Set 1 | neiafu
Processing Record 9 of Set 1 | vaini
Processing Record 10 of Set 1 | busselton
Processing Record 11 of Set 1 | talnakh
Processing Record 12 of Set 1 | punta arenas
Processing Record 13 of Set 1 | khatanga
Processing Record 14 of Set 1 | kavaratti
Processing Record 15 of Set 1 | castro
Processing Record 16 of Set 1 | hilo
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | atuona
Processing Record 19 of Set 1 | grand gaube
Processing Record 20 of Set 1 | puerto carreno
Processing Record 21 of Set 1 | dikson
Processing Record 22 of Set 1 |

In [15]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

701

## Add the weather data to a new DataFrame.

In [16]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [17]:
# Display sample data
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,College,US,64.8569,-147.8028,27.59,93,100,3.44,mist
1,Bonfim,BR,3.0833,-59.9500,90.16,52,100,5.06,overcast clouds
2,Ushuaia,AR,-54.8000,-68.3000,62.22,71,75,11.50,broken clouds
3,Hobart,AU,-42.8794,147.3294,44.46,91,100,12.66,shower rain
4,Esperance,AU,-33.8667,121.9000,54.18,85,33,8.70,scattered clouds
5,Neiafu,TO,-18.6500,-173.9833,74.08,100,75,0.00,light rain
6,Vaini,TO,-21.2000,-175.2000,77.16,88,40,6.91,scattered clouds
7,Busselton,AU,-33.6500,115.3333,51.84,82,0,11.88,clear sky
8,Talnakh,RU,69.4865,88.3972,-24.21,87,31,4.05,scattered clouds
9,Punta Arenas,CL,-53.1500,-70.9167,60.82,47,75,12.66,broken clouds


In [18]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Current Description'],
      dtype='object')

In [19]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"

# Recreate the DataFrame by using the new column order
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

city_data_df = city_data_df[["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,College,US,64.8569,-147.8028,27.59,93,100,3.44,mist
1,Bonfim,BR,3.0833,-59.9500,90.16,52,100,5.06,overcast clouds
2,Ushuaia,AR,-54.8000,-68.3000,62.22,71,75,11.50,broken clouds
3,Hobart,AU,-42.8794,147.3294,44.46,91,100,12.66,shower rain
4,Esperance,AU,-33.8667,121.9000,54.18,85,33,8.70,scattered clouds
5,Neiafu,TO,-18.6500,-173.9833,74.08,100,75,0.00,light rain
6,Vaini,TO,-21.2000,-175.2000,77.16,88,40,6.91,scattered clouds
7,Busselton,AU,-33.6500,115.3333,51.84,82,0,11.88,clear sky
8,Talnakh,RU,69.4865,88.3972,-24.21,87,31,4.05,scattered clouds
9,Punta Arenas,CL,-53.1500,-70.9167,60.82,47,75,12.66,broken clouds


In [20]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [23]:
# Set the output file name
output_data_file = "WeatherPy_Database.csv"
# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(output_data_file)